In [1]:
# http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#association-object
import sqlalchemy
sqlalchemy.__version__ 

'1.2.4'

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [3]:
from sqlalchemy import Column, Integer, String
from sqlalchemy import Table, ForeignKey
from sqlalchemy.orm import relationship

class Association(Base):
    __tablename__ = 'association'
    left_id = Column(Integer, ForeignKey('left.id'), primary_key=True)
    right_id = Column(Integer, ForeignKey('right.id'), primary_key=True)
    extra_data = Column(String(50))
    child = relationship("Child", back_populates="parents")
    parent = relationship("Parent", back_populates="children")

class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    children = relationship("Association", back_populates="parent")
    
    def __repr__(self):
        return f'<Parent {self.id}>'

class Child(Base):
    __tablename__ = 'right'
    id = Column(Integer, primary_key=True)
    parents = relationship("Association", back_populates="child")
    
    def __repr__(self):
        return f'<Child {self.id}>'
    
Base.metadata.create_all(engine)

2018-03-09 10:25:58,733 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-09 10:25:58,734 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,735 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-09 10:25:58,736 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,737 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("association")
2018-03-09 10:25:58,738 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,739 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("left")
2018-03-09 10:25:58,740 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,741 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("right")
2018-03-09 10:25:58,741 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,743 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "left" (
	id INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2018-03-09 10:25:58,743 INFO sqlalchemy.engine.base.Engine ()


In [4]:
s = Session()

for parent in [Parent(id = 10), Parent(id = 20)]:
    children = [Child(id = parent.id + i) for i in range(1, 5)]
    for child in children:
        ass = Association(extra_data="HELLO WORLD")  # (_*_) 
        ass.child = child
        ass.parent = parent
        s.add(ass)
        s.add(child)
        s.add(parent)
        
s.commit()

2018-03-09 10:25:58,780 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-09 10:25:58,781 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (?)
2018-03-09 10:25:58,782 INFO sqlalchemy.engine.base.Engine ((11,), (12,), (13,), (14,), (21,), (22,), (23,), (24,))
2018-03-09 10:25:58,784 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" (id) VALUES (?)
2018-03-09 10:25:58,784 INFO sqlalchemy.engine.base.Engine ((10,), (20,))
2018-03-09 10:25:58,787 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id, extra_data) VALUES (?, ?, ?)
2018-03-09 10:25:58,788 INFO sqlalchemy.engine.base.Engine ((10, 11, 'HELLO WORLD'), (10, 12, 'HELLO WORLD'), (10, 13, 'HELLO WORLD'), (10, 14, 'HELLO WORLD'), (20, 21, 'HELLO WORLD'), (20, 22, 'HELLO WORLD'), (20, 23, 'HELLO WORLD'), (20, 24, 'HELLO WORLD'))
2018-03-09 10:25:58,791 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
# N+1 (11) sql queries
s = Session()
ps = s.query(Parent).all()
for p in ps:
    for ass in p.children:
        print('>>>>>>', ass.child)
s.commit()

2018-03-09 10:25:58,800 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-09 10:25:58,801 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left"
2018-03-09 10:25:58,802 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,805 INFO sqlalchemy.engine.base.Engine SELECT association.left_id AS association_left_id, association.right_id AS association_right_id, association.extra_data AS association_extra_data 
FROM association 
WHERE ? = association.left_id
2018-03-09 10:25:58,806 INFO sqlalchemy.engine.base.Engine (10,)
2018-03-09 10:25:58,808 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = ?
2018-03-09 10:25:58,808 INFO sqlalchemy.engine.base.Engine (11,)
>>>>>> <Child 11>
2018-03-09 10:25:58,810 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = ?
2018-03-09 10:25:58,811 INFO sqlalchemy.engine.base.Engine (12,)
>>>>>> <Child 12>
2018-03-09 10:25:58,813 INF

In [12]:
s = Session()
from sqlalchemy.orm import subqueryload, lazyload
ps = s.query(Parent)\
      .options(subqueryload(Parent.children).subqueryload(Association.child)) \
      .filter(Association.extra_data != 'abcdef') \
      .all()
for p in ps:
    print('========', p)
    for ass in p.children:
        print('########', ass.child)
s.commit()

2018-03-09 10:29:27,379 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-09 10:29:27,380 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left", association 
WHERE association.extra_data != ?
2018-03-09 10:29:27,381 INFO sqlalchemy.engine.base.Engine ('abcdef',)
2018-03-09 10:29:27,385 INFO sqlalchemy.engine.base.Engine SELECT association.left_id AS association_left_id, association.right_id AS association_right_id, association.extra_data AS association_extra_data, anon_1.left_id AS anon_1_left_id 
FROM (SELECT "left".id AS left_id 
FROM "left", association 
WHERE association.extra_data != ?) AS anon_1 JOIN association ON anon_1.left_id = association.left_id ORDER BY anon_1.left_id
2018-03-09 10:29:27,385 INFO sqlalchemy.engine.base.Engine ('abcdef',)
2018-03-09 10:29:27,388 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id, association_1.right_id AS association_1_right_id 
FROM (SELECT "left".id AS left_id 
FROM "left", association 
WHERE 

In [7]:
s = Session()
from sqlalchemy.orm import subqueryload, lazyload
ps = s.query(Parent).options(lazyload(Parent.children).subqueryload(Association.child)).all()
for p in ps:
    print('========', p)
    for ass in p.children:
        print('########', ass.child)
s.commit()

2018-03-09 10:25:58,885 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-09 10:25:58,886 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left"
2018-03-09 10:25:58,887 INFO sqlalchemy.engine.base.Engine ()
======== <Parent 10>
2018-03-09 10:25:58,891 INFO sqlalchemy.engine.base.Engine SELECT association.left_id AS association_left_id, association.right_id AS association_right_id, association.extra_data AS association_extra_data 
FROM association 
WHERE ? = association.left_id
2018-03-09 10:25:58,892 INFO sqlalchemy.engine.base.Engine (10,)
2018-03-09 10:25:58,894 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id, anon_1.association_right_id AS anon_1_association_right_id 
FROM (SELECT DISTINCT association.right_id AS association_right_id 
FROM association 
WHERE ? = association.left_id) AS anon_1 JOIN "right" ON "right".id = anon_1.association_right_id ORDER BY anon_1.association_right_id
2018-03-09 10:25:58,895 INFO sqlalchemy.engine.base

In [8]:
s = Session()
from sqlalchemy.orm import subqueryload, lazyload
ps = s.query(Parent).options(subqueryload(Parent.children).lazyload(Association.child)).all()
for p in ps:
    print('========', p)
    for ass in p.children:
        print('########', ass.child)
s.commit()

2018-03-09 10:25:58,913 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-09 10:25:58,918 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left"
2018-03-09 10:25:58,919 INFO sqlalchemy.engine.base.Engine ()
2018-03-09 10:25:58,922 INFO sqlalchemy.engine.base.Engine SELECT association.left_id AS association_left_id, association.right_id AS association_right_id, association.extra_data AS association_extra_data, anon_1.left_id AS anon_1_left_id 
FROM (SELECT "left".id AS left_id 
FROM "left") AS anon_1 JOIN association ON anon_1.left_id = association.left_id ORDER BY anon_1.left_id
2018-03-09 10:25:58,923 INFO sqlalchemy.engine.base.Engine ()
======== <Parent 10>
2018-03-09 10:25:58,927 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = ?
2018-03-09 10:25:58,928 INFO sqlalchemy.engine.base.Engine (11,)
######## <Child 11>
2018-03-09 10:25:58,931 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
